In [1]:
# Knotion
# Ravelry + Notion integration

import requests
import json
import pandas as pd

In [353]:
config_path = "config.json"
with open(config_path, 'r') as configfile:
    config = json.load(configfile)

KNOTION_TOKEN = config['auth']['KNOTION_TOKEN']
DATABASE_ID = config['auth']['DATABASE_ID']

RAVELRY_BASIC_AUTH_USERNAME = config['auth']['RAVELRY_BASIC_AUTH_USERNAME']
RAVELRY_BASIC_AUTH_KEY = config['auth']['RAVELRY_BASIC_AUTH_KEY']
RAVELRY_USERNAME = config['auth']['RAVELRY_USERNAME']

# Notion API

In [361]:
class NotionDB:
    def __init__(self,DATABASE_ID,KNOTION_TOKEN):        
        self.DATABASE_ID = DATABASE_ID
        self.KNOTION_TOKEN = KNOTION_TOKEN

        headers = {
            "Authorization": "Bearer " + KNOTION_TOKEN,
            "accept": "application/json",
            "Notion-Version": "2022-06-28",
            "content-type": "application/json",

        }

        self.headers = headers

        def queryNotionDatabase(DATABASE_ID,KNOTION_TOKEN, headers):
            readURL = f'https://api.notion.com/v1/databases/{DATABASE_ID}/query'
            return requests.request("POST", readUrl, headers=headers)

        def readNotionDatabase( DATABASE_ID,KNOTION_TOKEN,headers):
            readURL = f'https://api.notion.com/v1/databases/{DATABASE_ID}'
            return requests.request("GET", readURL, headers=headers)


        self.db = queryNotionDatabase(DATABASE_ID,KNOTION_TOKEN,headers)
        self.meta = readNotionDatabase(DATABASE_ID,KNOTION_TOKEN,headers)

    def to_pandas(self):
        r_json = notion.db.json()
        self.df = pd.DataFrame(pd.json_normalize(r_json['results']))
        return self.df

    def check_notion(self):
        # check if knotion db is already set up (has the correct properties)
        if not hasattr(self, 'properties'):
            self.get_properties()
        prop_current = notion.properties
        prop_missing = {}
        for k in config['properties'].keys():
            if k in notion.properties.keys():
                if  notion.properties[k]['type'] != config['properties'][k]['type']:
                    prop_missing[k] = config['properties'][k]
                    prop_missing[k]['description'] = "type"
            else:
                prop_missing[k] = config['properties'][k]
                prop_missing[k]['description'] = "property"
        return prop_missing

    def setup_notion(self):
        # set up knotion Notion Database
        print(self.db)

    def get_properties(self, output=None):
        prop = self.meta.json()['properties']
        if output == "json":
            self.properties = prop
            return prop
        else:
            prop_dict = {}
            for prop_name in prop.keys():
                prop_dict[prop_name] = {'type':prop[prop_name]['type']}
            self.properties = prop_dict
            return prop_dict

    def add_new_property(self,properties):
        url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/"
        payload = {
                "properties": properties
            }

        return requests.patch(url, data=json.dumps(payload), headers=self.headers)

    def setup_add_property_payload():
        prop_missing = self.check_notion()
        properties = {}

        for k in prop_missing.keys():
            if prop_missing[k]['description'] == 'property':
                properties[k] = {'type': prop_missing[k]["type"], prop_missing[k]["type"] :{}}

        return properties



In [367]:
notion = NotionDB(DATABASE_ID,KNOTION_TOKEN)


In [368]:
prop_missing = notion.check_notion()
properties = {}

for k in prop_missing.keys():
    if prop_missing[k]['description'] == 'property':
        properties[k] = {'type': prop_missing[k]["type"], prop_missing[k]["type"] :{}}
properties

{}

In [365]:
notion.add_new_property(properties)

<Response [200]>

In [377]:
properties = {
    "appID": {
                "rich_text": [
                    {
                        "text": {
                            "content": "Amazing"
                        }
                    }
                ]
            },
    "Yarn Name": {
                "title": [
                    {
                        "text": {
                            "content": "New DK Yarn"
                        }
                    }
                ]
            },
    "Tags": {
                "multi_select": [
                    {
                        "name": "Dk"
                    }
                ]
            }
    
  }

In [374]:
properties

{'appID': {'rich_text': [{'text': {'content': 'Amazing'}}]},
 'Yarn Name': {'title': [{'text': {'content': 'New DK Yarn'}}]},
 'Tags': {'multi_select': [{'name': {'Dk'}}]}}

In [378]:
url = 'https://api.notion.com/v1/pages'

payload = {
    "parent": { "database_id": DATABASE_ID},
    "properties": properties
}

res = requests.request("POST", url, data=json.dumps(payload), headers=headers)

print(res.status_code)
print(res.text)

200
{"object":"page","id":"d3aedcca-733b-4df5-8f9e-2fac3c7e3c3f","created_time":"2023-01-29T22:36:00.000Z","last_edited_time":"2023-01-29T22:36:00.000Z","created_by":{"object":"user","id":"eed2cf11-82bd-4aed-8d4a-7e51a8e9fb21"},"last_edited_by":{"object":"user","id":"eed2cf11-82bd-4aed-8d4a-7e51a8e9fb21"},"cover":null,"icon":null,"parent":{"type":"database_id","database_id":"fbf11eb2-46d2-4d6a-89a7-53e48a0b2bcf"},"archived":false,"properties":{"Another Property":{"id":"NE%7B%3C","type":"rich_text","rich_text":[]},"MultiAdd TEST 2":{"id":"NRl%3E","type":"status","status":null},"Another TEST Property":{"id":"R%5B%60s","type":"rich_text","rich_text":[]},"Yarn Company":{"id":"YhJ~","type":"rich_text","rich_text":[]},"new text":{"id":"_vS%60","type":"number","number":null},"New Property":{"id":"%60gFF","type":"rich_text","rich_text":[]},"new text - rich":{"id":"aaLW","type":"rich_text","rich_text":[]},"new property":{"id":"b%3BI%5C","type":"number","number":null},"MultiAdd Test 1":{"id":"ec

# Ravelry API

In [75]:
class RavelryDB:
    def __init__(self,RAVELRY_BASIC_AUTH_USERNAME,RAVELRY_BASIC_AUTH_KEY,RAVELRY_USERNAME):
        def readRavelryProjectDatabase(RAVELRY_BASIC_AUTH_USERNAME,RAVELRY_BASIC_AUTH_KEY,RAVELRY_USERNAME):
            base_url = 'https://api.ravelry.com'
            return requests.get(base_url+f'/projects/{RAVELRY_USERNAME}/list.json?include=collections', auth = (RAVELRY_BASIC_AUTH_USERNAME,RAVELRY_BASIC_AUTH_KEY))

        def readRavelryStashDatabase(RAVELRY_BASIC_AUTH_USERNAME,RAVELRY_BASIC_AUTH_KEY,RAVELRY_USERNAME):
            base_url = 'https://api.ravelry.com'
            return requests.get(base_url+f'/people/{RAVELRY_USERNAME}/stash/list.json?include=collections', auth = (RAVELRY_BASIC_AUTH_USERNAME,RAVELRY_BASIC_AUTH_KEY))

        self.RAVELRY_BASIC_AUTH_USERNAME = RAVELRY_BASIC_AUTH_USERNAME
        self.RAVELRY_BASIC_AUTH_KEY = RAVELRY_BASIC_AUTH_KEY
        self.RAVELRY_USERNAME = RAVELRY_USERNAME
        self.projects   = readRavelryProjectDatabase(RAVELRY_BASIC_AUTH_USERNAME,RAVELRY_BASIC_AUTH_KEY,RAVELRY_USERNAME)
        self.stash = readRavelryStashDatabase(RAVELRY_BASIC_AUTH_USERNAME,RAVELRY_BASIC_AUTH_KEY,RAVELRY_USERNAME)
        self.to_pandas()

    def to_pandas(self):
        r_json = self.projects.json()
        self.project_df = pd.DataFrame(pd.json_normalize(r_json['projects']))
        r_json = self.stash.json()
        self.stash_df = pd.DataFrame(pd.json_normalize(r_json['stash']))
        return self.project_df, self.stash_df

    # PROJECT Functions
    def get_project_ids(self):
        if not hasattr(self, 'project_df'):
            self.to_pandas()
        return [id for id in self.project_df.id]
    
    def get_project_names(self):
        if not hasattr(self, 'project_df'):
            self.to_pandas()
        return [id for id in self.project_df.name]

    def get_project_full_single(self,id):
        PROJECT_ID = id #self.project_df.id[id]
        base_url = 'https://api.ravelry.com'
        project_r = requests.get(base_url+f'/projects/{RAVELRY_USERNAME}/{PROJECT_ID}.json', auth = (RAVELRY_BASIC_AUTH_USERNAME,RAVELRY_BASIC_AUTH_KEY))
        r_json = project_r.json()
        return pd.DataFrame(pd.json_normalize(r_json))

    def get_project_full(self):
        project_df_list = [self.get_project_full_single(PROJECT_ID) for PROJECT_ID in self.get_project_ids()]
        self.project_df_full = pd.concat(project_df_list)
        return self.project_df_full.head()

    # STASH functions
    def get_stash_ids(self):
        if not hasattr(self, 'stash_df'):
            self.to_pandas()
        return [id for id in self.stash_df.id]
        
    def get_stash_names(self):
        if not hasattr(self, 'stash_df'):
            self.to_pandas()
        return [id for id in self.stash_df.name]

    def get_stash_full_single(self,id):
        STASH_ID = id
        base_url = 'https://api.ravelry.com'
        project_r = requests.get(base_url+f'/people/{RAVELRY_USERNAME}/stash/{STASH_ID}.json', auth = (RAVELRY_BASIC_AUTH_USERNAME,RAVELRY_BASIC_AUTH_KEY))
        r_json = project_r.json()
        return pd.DataFrame(pd.json_normalize(r_json))
    
    def get_stash_full(self):
        stash_df_list = [self.get_stash_full_single(STASH_ID) for STASH_ID in self.get_stash_ids()]
        self.stash_df_full = pd.concat(stash_df_list)
        return self.stash_df_full.head()

    def parse_stash_pack(self,id):
        if not hasattr(self, 'stash_df_full'):
            self.get_stash_full()
        return pd.DataFrame(ravelry.stash_df_full['stash.packs'][ravelry.stash_df_full['stash.id']==id].values[0])

In [337]:
ravelry = RavelryDB(RAVELRY_BASIC_AUTH_USERNAME,RAVELRY_BASIC_AUTH_KEY,RAVELRY_USERNAME)
ravelry

In [338]:
id = ravelry.get_stash_ids()[24]
pack = ravelry.parse_stash_pack(id)

In [90]:
pack

,id,primary_pack_id,project_id,skeins,stash_id,total_grams,total_meters,total_ounces,total_yards,yarn_id,...,color_family_id,grams_per_skein,yards_per_skein,meters_per_skein,ounces_per_skein,prefer_metric_weight,prefer_metric_length,shop_id,thread_size,color_attributes
0,103733928,NaN,NaN,2.00,24826729,200.0,640.1,7.05,700.0,192638,...,None,100.0,350.0,320.0,3.53,True,False,None,None,[]
1,103733929,103733928.0,NaN,1.82,24826729,200.0,582.9,7.05,637.5,192638,...,None,NaN,NaN,NaN,NaN,True,False,None,None,[]
2,108910918,103733928.0,30795153.0,NaN,24826729,NaN,NaN,NaN,NaN,192638,...,None,100.0,350.0,320.0,3.53,True,False,None,None,[]
3,104640040,103733928.0,29684683.0,0.18,24826729,NaN,57.2,NaN,62.5,192638,...,None,100.0,350.0,320.0,3.53,True,False,None,None,[]


In [342]:
notion.meta.json()['properties']

{'Another Property': {'id': 'NE%7B%3C',
  'name': 'Another Property',
  'type': 'rich_text',
  'rich_text': {}},
 'MultiAdd TEST 2': {'id': 'NRl%3E',
  'name': 'MultiAdd TEST 2',
  'type': 'status',
  'status': {'options': [], 'groups': []}},
 'Another TEST Property': {'id': 'R%5B%60s',
  'name': 'Another TEST Property',
  'type': 'rich_text',
  'rich_text': {}},
 'new text': {'id': '_vS%60',
  'name': 'new text',
  'type': 'number',
  'number': {'format': 'number'}},
 'New Property': {'id': '%60gFF',
  'name': 'New Property',
  'type': 'rich_text',
  'rich_text': {}},
 'new text - rich': {'id': 'aaLW',
  'name': 'new text - rich',
  'type': 'rich_text',
  'rich_text': {}},
 'new property': {'id': 'b%3BI%5C',
  'name': 'new property',
  'type': 'number',
  'number': {'format': 'number'}},
 'MultiAdd Test 1': {'id': 'eco%3C',
  'name': 'MultiAdd Test 1',
  'type': 'rich_text',
  'rich_text': {}},
 'Status': {'id': 'fFc%3B',
  'name': 'Status',
  'type': 'status',
  'status': {'options':